# Import Libraries

In [124]:
!pip install pandas

In [125]:
import pandas as pd
import re

# Combine dataset from scrape dataset

In [126]:
# Daftar path file CSV beserta label kategorinya
file_paths = {
    r"raw-dataset\Scrapcookpad(cleanfood).csv": "cleanfood",
    r"raw-dataset\Scrapcookpad(diet).csv": "diet",
    r"raw-dataset\Scrapcookpad(rendahkalori).csv": "rendahkalori",
    r"raw-dataset\Scrapcookpad(sehat).csv": "sehat"
}

In [127]:
# Membaca, memberi label, dan menggabungkan semua file CSV
dataframes = []
for path, label in file_paths.items():
    df = pd.read_csv(path)
    df['kategori'] = label  # Menambahkan kolom kategori
    dataframes.append(df)

In [128]:
# Gabungkan semua dataframe
combined_df = pd.concat(dataframes, ignore_index=True)

In [129]:
# Hapus duplikat berdasarkan judul dan url (jika ada)
combined_df.drop_duplicates(subset=['Judul', 'URL'], inplace=True)

In [130]:
output_path = r"recipes.csv"
combined_df.to_csv(output_path, index=False)

In [131]:
print(f"Jumlah baris: {combined_df.shape[0]}")
print(f"Jumlah kolom: {combined_df.shape[1]}")

Jumlah baris: 11934
Jumlah kolom: 5


# Preprocessing

In [132]:
df = pd.read_csv('recipes.csv')
df.head()

,Judul,URL,Bahan,Langkah,kategori
0,Steak Ayam Lemon Madu (Eat Clean Series),https://cookpad.com/id/resep/1419994-steak-aya...,1/2 kg daging dada ayam tanpa tulang; 1/2 buah...,1\n \n\n\n\nKucuri daging ayam dengan sedik...,cleanfood
1,Bala bala Oatmeal Clean eating,https://cookpad.com/id/resep/14034644-bala-bal...,4 sdm oatmeal instan; 1 buah wortel iris korek...,1\n \n\n\n\nCampur semua bahan jadi satu ta...,cleanfood
2,Chicken steak (menu diet/clean eating),https://cookpad.com/id/resep/9595903-chicken-s...,200 gr ayam kampung fillet; Brokoli; Wortel; S...,1\n \n\n\n\nRebus ayam sebentar sampai sete...,cleanfood
3,Clean Eating Aini'Cooking #2,https://cookpad.com/id/resep/23944711-clean-ea...,1 buah wortel kecil potong dadu; 1 buah jagung...,1\n \n\n\n\nSiapkan semua siapkan semua bah...,cleanfood
4,Clean Eating Aini'Cooking #4,https://cookpad.com/id/resep/23961936-clean-ea...,Secukupnya ayam yang telah dimarinasi semalama...,1\n \n\n\n\nRebus wortel dan brokoli lalu t...,cleanfood


In [133]:
# Langkah jadi list, hapus newline dan strip
def clean_steps(raw_text):
    # Pisahkan berdasarkan angka di awal langkah (1\n, 2\n, dst.)
    steps = re.split(r'\n?\s*\d+\s*\n+', raw_text)
    # Bersihkan spasi kosong dan newline
    cleaned = [re.sub(r'\s+', ' ', step).strip() for step in steps if step.strip()]
    return cleaned

df['list_langkah'] = df['Langkah'].apply(clean_steps)

In [134]:
# Bahan jadi list
df['list_bahan'] = df['Bahan'].str.split(';').apply(lambda x: [b.strip() for b in x if b.strip()])

def clean_bahan(bahan_list):
    satuan = r'(kg|gram|gr|sdt|sdm|buah|siung|lembar|ml|liter|ltr|butir|sendok|cup|potong|iris|secukupnya)'
    cleaned = []
    for b in bahan_list:
        # Hilangkan angka, pecahan, dan satuan
        b_clean = re.sub(r'\b\d+(/\d+)?\b', '', b)  # hapus angka & pecahan
        b_clean = re.sub(satuan, '', b_clean, flags=re.IGNORECASE)
        b_clean = re.sub(r'\s+', ' ', b_clean).strip()  # hapus spasi berlebih
        cleaned.append(b_clean)
    return cleaned

df['label_bahan'] = df['list_bahan'].apply(clean_bahan)

In [135]:
# Peta metode masak beserta sinonimnya
metode_map = {
    'panggang': r'panggang|dipanggang|oven|grill|bakar',
    'rebus': r'rebus|mendidih|didihkan|dididihkan',
    'kukus': r'kukus|steam|dikukus',
    'goreng': r'goreng|digoreng|deep fry|fried|orak arik',
    'tumis': r'tumis|saute|ditumis|menumis',
    'ungkep': r'ungkep|ungkepan',
    'masak': r'masak|memasak|dimasak|panaskan|dimasukkan ke dalam wajan',
    'simpan dingin': r'simpan di kulkas|dinginkan|overnight|semalaman|kulkas',
    'blender': r'blender|haluskan|dihaluskan|lumatkan|blend',
    'campur': r'campur|aduk|tuang|mix|diaduk',
}

def ekstrak_metode_memasak(langkah_list):
    hasil = set()

    for langkah in langkah_list:
        langkah_lower = langkah.lower()

        # Matching keyword
        for metode, pattern in metode_map.items():
            if re.search(pattern, langkah_lower):
                hasil.add(metode)

        # Rule-based tambahan (jika perlu tambahkan di sini)
        if 'air mendidih' in langkah_lower and 'masukkan' in langkah_lower:
            hasil.add('rebus')
        if 'tutup dan simpan' in langkah_lower and 'kulkas' in langkah_lower:
            hasil.add('simpan dingin')
    
    return list(hasil)

df['metode_memasak'] = df['list_langkah'].apply(ekstrak_metode_memasak)

In [136]:
# Hapus kolom yang tidak dibutuhkan
df.drop(columns=['URL', 'Bahan', 'Langkah'], inplace=True)

# Hapus baris yang mengandung data kosong (NaN)
df.dropna(inplace=True)

# Buat mask: True untuk baris dengan metode_memasak kosong (list kosong)
kosong_mask = df['metode_memasak'].apply(lambda x: isinstance(x, list) and len(x) == 0)

# Tampilkan baris yang akan dihapus (opsional, untuk cek)
baris_kosong = df[kosong_mask]
print("Jumlah baris dengan metode_memasak kosong:", len(baris_kosong))

# Hapus baris tersebut dari df
df = df[~kosong_mask].reset_index(drop=True)

# Cek hasil
df.head()

Jumlah baris dengan metode_memasak kosong: 237


,Judul,kategori,list_langkah,list_bahan,label_bahan,metode_memasak
0,Steak Ayam Lemon Madu (Eat Clean Series),cleanfood,"[Kucuri daging ayam dengan sedikit air lemon, ...","[1/2 kg daging dada ayam tanpa tulang, 1/2 bua...","[daging dada ayam tanpa tulang, perasan jeruk ...","[masak, panggang, campur]"
1,Bala bala Oatmeal Clean eating,cleanfood,[Campur semua bahan jadi satu tambahkan air se...,"[4 sdm oatmeal instan, 1 buah wortel iris kore...","[oatmeal instan, wortel korek api, kol, batang...",[campur]
2,Chicken steak (menu diet/clean eating),cleanfood,"[Rebus ayam sebentar sampai setengah matang, t...","[200 gr ayam kampung fillet, Brokoli, Wortel, ...","[ayam kampung fillet, Brokoli, Wortel, Slada, ...","[kukus, rebus, masak]"
3,Clean Eating Aini'Cooking #2,cleanfood,"[Siapkan semua siapkan semua bahan, cuci bersi...","[1 buah wortel kecil potong dadu, 1 buah jagun...","[wortel kecil dadu, jagung lalu serut, tomat j...",[rebus]
4,Clean Eating Aini'Cooking #4,cleanfood,"[Rebus wortel dan brokoli lalu tiriskan. |, Pa...",[Secukupnya ayam yang telah dimarinasi semalam...,"[ayam yang telah dimarinasi semalaman, Brokoli...","[tumis, rebus, masak]"


# EDA

In [137]:
# Jumlah resep per kategori
print(df['kategori'].value_counts())

# Frekuensi metode memasak
from collections import Counter
methods = df['metode_memasak'].explode()
print(Counter(methods))

kategori
diet            5795
sehat           3850
rendahkalori    1795
cleanfood        257
Name: count, dtype: int64
Counter({'campur': 8466, 'masak': 5208, 'rebus': 4218, 'blender': 4001, 'tumis': 3117, 'panggang': 2530, 'goreng': 2299, 'simpan dingin': 1646, 'kukus': 1463, 'ungkep': 63})
